In [10]:
import pathlib

import torch

from pylon_experiments.model.model import NextActivityPredictor, Args

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint = pathlib.Path("../runs/bpic2012/20250212.2112.no_constraint/model.best_val_loss.pth")
model = torch.load(checkpoint, weights_only=False).to(torch.device("cpu"))
model

NextActivityPredictor(
  (embedding): Embedding(40, 128, padding_idx=0)
  (lstm): LSTM(128, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=40, bias=True)
)

In [11]:
from tqdm import tqdm

from pylon_experiments.data.loader import Loader, Args as LoaderArgs

train, val = Loader(args=LoaderArgs(
    dataset_path="../data/bpic2012",
    batch_size=2,
)).get_loaders().values()

train

ValueError: too many values to unpack (expected 2)

In [ ]:
loaders[4]

TypeError: 'dict_values' object is not subscriptable

In [ ]:
for l in loaders:
    print(l.dataset)
    print(l.dataset[0])

(tensor([2]), tensor(4), tensor(1))
(tensor([2]), tensor(4), tensor(1))
(tensor([2]), tensor(4), tensor(1))
(tensor([ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 16, 17, 16,
        17, 16, 17, 16, 17, 16, 17, 16, 17, 16, 36, 26, 17,  3]), tensor(32, dtype=torch.uint8))
(tensor([ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 16, 18, 19,
        17, 20, 25, 20, 30, 25, 31, 32, 31, 19, 32, 20, 21, 22, 24, 23, 25,  3]), tensor(36, dtype=torch.uint8))
(tensor([ 2,  4,  5,  6,  7,  8,  9, 11, 10, 12, 13, 14, 15, 16, 17, 16, 17, 16,
        26, 36, 17,  3]), tensor(22, dtype=torch.uint8))


In [ ]:
from itertools import zip_longest


loader = tqdm(
    loaders[4],
    unit="batch",
    bar_format="{l_bar}{bar:10}{r_bar}{bar:-10b}",
)

count = 0
total_correct = 0
total_samples = 0
for trace, lengths in loader:
    batch_logits = []

    x = trace[:, :6]
    for i in range(6, lengths[0]):
        lengths = torch.tensor([i] * len(x))
        logits = model(x, lengths)
        batch_logits.append(logits)
        for j in range(len(x)):
            correct = logits[j].argmax(dim=-1) == trace[j, i]
            correct = 'correct' if correct else 'error'
            print(f"[{correct}][len: {lengths[j].item()}] {x[j].numpy()} -> {logits.argmax(dim=-1)[j]} ({trace[j, i].item()})")
        x = torch.cat([x, logits.argmax(dim=1, keepdim=True)], dim=1)
        
        print((logits[0].argmax(dim=-1) == trace[0, i]))
        total_correct += (logits.argmacx(dim=-1) == trace[:, i]).sum().item()
        total_samples += len(trace)
    break
        

print(total_correct / total_samples)            

  0%|          | 0/234 [00:00<?, ?batch/s]

[error][len: 6] [2 4 5 6 7 8] -> 15 (9)
[correct][len: 6] [2 4 5 6 7 8] -> 15 (15)
tensor(False)


AttributeError: 'Tensor' object has no attribute 'argmacx'